# Libraries

In [ ]:
!pip install rouge_score
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!pip install datasets transformers[sentencepiece]
!pip install FLAX
!pip install sentencepiece
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
os.chdir('/content/drive/MyDrive/NEU/IE7500/Project_IE7500')

In [ ]:
#import tensorflow as tf
#import tensorflow_datasets as tfds

import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from ast import literal_eval
import re
import math
from tqdm import tqdm

#from bs4 import BeautifulSoup 
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt 

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from transformers import pipeline, AutoTokenizer

In [ ]:
import time
a = time.time()

In [ ]:
from datasets import load_dataset, load_metric

# Data

Due to time it takes, tensorflow data was downloaded and saved as csv for iteration process to take less time

In [ ]:
ds = load_dataset('cnn_dailymail', '3.0.0')
ds

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

# Model predictions

## 1. T5

In [ ]:
from transformers import T5Tokenizer, FlaxT5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = FlaxT5ForConditionalGeneration.from_pretrained("t5-small")

In [ ]:
# Add prefix
df_test = ["summarize: " + doc for doc in ds["test"]["article"]]

In [ ]:
# Generate Summary
n_sample = 100
summary_ids = [model.generate(tokenizer(t, return_tensors="np", max_length=1024, truncation=True)["input_ids"]).sequences for t in tqdm(df_test[0:n_sample])]
#print(tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False))

 98%|█████████▊| 98/100 [20:15<00:24, 12.43s/it]

In [ ]:
summary_df = []
for i in tqdm(range(len(summary_ids))):
  summary_df.append(tokenizer.batch_decode(summary_ids[i], skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])

100%|██████████| 100/100 [00:01<00:00, 69.41it/s]


In [ ]:
y_test = ds["test"]["highlights"][0:n_sample]
y_pred = pd.Series(summary_df)

In [ ]:
import evaluate
rouge_score = evaluate.load("rouge")
rouge_score.compute(predictions=y_pred, references=y_test)


{'rouge1': 0.24147191823398717,
 'rouge2': 0.08869545909303395,
 'rougeL': 0.19176589578531436,
 'rougeLsum': 0.2164042315558265}

## 2. BART CNN

In [ ]:
model_type ="facebook/bart-large-cnn"

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")


In [ ]:
summary_ids = [model.generate(tokenizer(t, return_tensors="pt", max_length=1024, truncation=True)["input_ids"]) for t in tqdm(ds["test"]["article"][0:n_sample])]


In [ ]:
summary_df_bart = []
for i in tqdm(range(len(summary_ids))):
  summary_df_bart.append(tokenizer.batch_decode(summary_ids[i], skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])

In [ ]:
y_test = ds["test"]["highlights"][0:n_sample]
y_pred = pd.Series(summary_df_bart)

In [ ]:
import evaluate
rouge_score = evaluate.load("rouge")
rouge_score.compute(predictions=y_pred, references=y_test)


In [ ]:
y_test[0]

In [ ]:
y_pred[0]